In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy.stats as stats

from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu

import statsmodels.api as sm
from statsmodels.formula.api import ols

# union of all measurements across multiple(3) days
# return a list of values
def union_all_days(df,key): 
    df1 = df[key]
    a=[]
    for header in list(df1.columns):
        a += list(df1[header].dropna())
    return a

In [6]:
# Statistical analysis of the effect of NO (SNP treatment) on high CO2 induced stomatal closure

sheetname1 = 'Fig_7A' # Figure 7A - effect of NO (SNP treatment) on high CO2 induced stomatal closure

#below are data of other figures. The code for analysis may need adjustments (e.g. updating headers) to work on those data.
#sheetname1 = 'Fig_S1' # Figure S1 - effect of SNAP treatment on high CO2 induced stomatal closure
#sheetname1 = 'Fig_7B' # SNP can induce stomatal closure under ambient CO2
#sheetname1 = 'Fig_8A' # cPTIO (NO scavenger) can impair high CO2 induced stomatal closure

df = pd.read_excel('CO2_model_data_20231120.xlsx', header=[0,1,2,3], sheet_name=sheetname1)

# melting multiindex data
df1 = pd.melt(df, var_name=['Treatment','CO2','Time','Day'], value_name='aperture').dropna()
#df1.to_csv('CO2_model_data_20231120'+sheetname1+'_melted.csv',index=False, encoding='utf_8_sig')
df1

,Treatment,CO2,Time,Day,aperture
0,Preincubation,control(0 time),10min,Day1,2.841
1,Preincubation,control(0 time),10min,Day1,3.108
2,Preincubation,control(0 time),10min,Day1,2.728
3,Preincubation,control(0 time),10min,Day1,2.004
4,Preincubation,control(0 time),10min,Day1,1.201
...,...,...,...,...,...
2692,0.15mM SNP,800ppm,20min,Day3,2.043
2693,0.15mM SNP,800ppm,20min,Day3,2.246
2694,0.15mM SNP,800ppm,20min,Day3,1.287
2695,0.15mM SNP,800ppm,20min,Day3,1.724


In [7]:
# Student's t-test 
# Below are example tests for Fig.7A. To use these code on the data of other figures, please update the column names (headers)

# H0: In Col-0 under high CO2 and 10 min/20 min, aperture under 0.15mM SNP is similar to aperture without SNP
A1=('0.15mM SNP', '800ppm','10min')
A2=('0mM SNP', '800ppm','10min')
s1 = union_all_days(df,A1)
s2 = union_all_days(df,A2)
print (ttest_ind(s1, s2))

A1=('0.15mM SNP', '800ppm','20min')
A2=('0mM SNP', '800ppm','20min')
s1 = union_all_days(df,A1)
s2 = union_all_days(df,A2)
print (ttest_ind(s1, s2))

# conclusion: reject H0, i.e. SNP causes significant change of aperture size

Ttest_indResult(statistic=-7.9004495924385845, pvalue=6.2639125005572e-14)
Ttest_indResult(statistic=-7.668319893153531, pvalue=3.3857870548784204e-13)


E:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


In [8]:
# perform two-way ANOVA with interaction
df = pd.read_csv('CO2_model_data_20231120'+sheetname1+'_melted.csv')
print (sheetname1) # this is a sanity check that we're on the right sheet (i.e. right dataset)

df1=df[df['Treatment']!='Preincubation']
df2=df1[df1['Time']=='10min'] 
#df2=df1[df1['Time']=='20min']

model = ols('aperture ~ C(Treatment) + C(CO2) + C(Treatment):C(CO2)', data=df2).fit()
sm.stats.anova_lm(model, typ=2)

# conclusion: A significant interaction term between SNP treament and high CO2 indicate NO causes hypersensitivity in high CO2 induced stomatal closure

Fig_7A


,sum_sq,df,F,PR(>F)
C(Treatment),21.760311,2.0,30.625489,1.477612e-13
C(CO2),95.472331,1.0,268.735755,1.677762e-52
C(Treatment):C(CO2),2.872152,2.0,4.042270,1.790606e-02
Residual,293.448654,826.0,NaN,NaN
